In [ ]:
import os
import pandas as pd
import numpy as np
from decouple import config

In [ ]:
%run parser_web.py

In [ ]:
if os.path.exists("./abstracts_content.pkl"):
    # To read in data if already saved
    df = pd.read_pickle("abstracts_content.pkl")
else:
    %run parser_web.py

In [ ]:
# This is for downloading figures and references. Not necessary at this stage:
'''
apiKey = config('AI_API_KEY')

df['cited'] = None
df['figs'] = None

for index, row in df.iterrows():
    if row['abstractType'] == 'Abstract Submission':
        abstractId = row['submissionNumber']   
        try:            
            if not row['cited']: 
                response = requests.get("https://ww4.aievolution.com/hbm2101/?do=cnt.getservice&service=getAbstract&abstractID=%s&includeAbstractFormData=1&apiKey=%s" % (index, apiKey))
                res = response.text
                soup = Soup(res, 'xml')
                res_text = soup.prettify()
                o = xmltodict.parse(res_text)  
                for i in o['rst']['abstract']['forms']['form']['responses']['questions']['question']:
                    if i['@id'] == '2070':
                        df['cited'].iloc[int(index)] = i['answers']['#text']
                df.to_pickle("abstracts_content.pkl")
        except:
            a = 0
        
        figNum = int(row['figures']['@count'])
        try:
            if figNum > 0:
                if not row['figs']:                  
                    df['figs'].iloc[int(index)] = figNum
                    figDir = "figures/%i" % int(abstractId)
                    if not os.path.isdir(figDir):
                        os.mkdir(figDir)
                    for f in range(figNum):
                        URL = row['figures']['figure'][f]['imageFilename']
                        response = requests.get(URL)
                        file = open("figures/%i/Figure%i.png" % (int(abstractId), f + 1), "wb")
                        file.write(response.content)
                        file.close()
            # print(abstractId)
            df.to_pickle("abstracts_content.pkl")
        except:
            a = 1
'''


In [ ]:
# Filter by aceepted flag and add pimary category
df_accepted = df[(df["abstractType"] == "Abstract Submission") & (df["acceptedFlag"] == "Yes") & (df["completedFlag"] == "Yes")] # & df["cited"] & df["figs"]]
df_accepted['primary_category'] = None
for idx, row in df_accepted.iterrows():
    pc = row["categories"]['category'][-1]['name']
    df_accepted.loc[idx, 'primary_category'] = pc

In [ ]:
# keep needed columns
num = len(df_accepted)
# num = 50 # number of abstracts to run for test
print(num)
cols = ['submissionNumber', 'title', 'authors','institution', 'previewurl'] # "purpose","materialsMethods", "results","conclusion","cited","figs"
if num == len(df_accepted):
    df_filtered = df_accepted.loc[:, cols]
else:
    df_filtered = df_accepted.loc[df_accepted.index[:num], cols]
    df_accepted = df_accepted.loc[df_accepted.index[:num], :]
df_filtered.to_csv("2021_abstracts.csv", header=cols, index=False,sep="€")

!sed -i -e 's/<u>/\\underline\{/g' 2021_abstracts.csv
!sed -i -e 's/<\/u>/\}/g' 2021_abstracts.csv
!sed -i -e 's/<sup>/\\textsuperscript\{/g' 2021_abstracts.csv
!sed -i -e 's/<\/sup>/\}/g' 2021_abstracts.csv
!sed -i -e 's/\ \&\ /\ \\\&\ /g' 2021_abstracts.csv
!sed -i -e 's/\&E/\\\&E/g' 2021_abstracts.csv
!sed -i -e 's/A\&/A\\\&/g' 2021_abstracts.csv
!sed -i -e 's/\_/\\\_/g' 2021_abstracts.csv
!sed -i -e 's/\\n\\n/\\linebreak/g' 2021_abstracts.csv

!sed -i -e 's/\|/\\\|/g' 2021_abstracts.csv #CHECK ################
!sed -i -e 's/\n/\ /g' 2021_abstracts.csv #CHECK ################
!sed -i -e 's/.\€\\underline\{/\€\\underline\{/g' 2021_authors_index.csv
# TO ADD:
# REPLACE: , -- SELECT --, 
!rm -f 2021_abstracts.csv-e

In [ ]:
# create authro index reference 
authors = {}
author_no_resp = set()
check_these = []
for i, row in df_accepted.iterrows():
    for speaker in row['speakers']['speaker']:
        speaker_id = int(speaker['@id'])
        if speaker_id not in authors:
            authors[speaker_id] = {'firstname': speaker['firstname'], 
                                    'middlename': speaker['middlename'], 
                                    'lastname': speaker['lastname'], 
                                    'submissionNumber': set()}  # use set to prevent duplication
        if type(row['submissionNumber']) is list:
            for s in row['submissionNumber']:
                authors[speaker_id]['submissionNumber'].add(int(s))
        else:
            authors[speaker_id]['submissionNumber'].add(int(row['submissionNumber']))

In [ ]:
# create authors index 
def capitalise_name(x):    
    x = " ".join(i.capitalize() for i in x.split(" "))
    if len(x.split("-")) > 1:
        x = "-".join(i.capitalize() for i in x.split("-"))
    return x

with open("2021_authors_index.csv","w", encoding='utf8') as f:
    for author in authors.values(): 
        if author['middlename'] != None:  # no mi
            f.write("%s€%s %s€%s\n" % (capitalise_name(author['lastname']),
                                       capitalise_name(author['firstname']),
                                       capitalise_name(author['middlename']),
                                       str(list(author['submissionNumber']))))
        else:
            f.write("%s€%s€%s\n" % (capitalise_name(author['lastname']),
                                    capitalise_name(author['firstname']),
                                    str(list(author['submissionNumber']))))

!sed -i -e 's/\[//g'  2021_authors_index.csv
!sed -i -e 's/\]//g'  2021_authors_index.csv
!sed -i -e 's/Dr //g'  2021_authors_index.csv
!sed -i -e 's/\.//g'  2021_authors_index.csv
!sed -i -e 's/\.\ //g'  2021_authors_index.csv
!sed -i -e 's/\*//g'  2021_authors_index.csv
!sed -i -e 's/\,\ /\,/g' 2021_authors_index.csv # replace space after commas, BUT ONLY FOR NUMBERS !!!

!sort -k1 -t\| -o 2021_authors_index.csv 2021_authors_index.csv 
# !sed -i -e 's/\\|/\\€/g'  2021_authors_index.csv
!rm -f 2021_authors_index.csv-e

In [ ]:
# create category index

df = pd.read_csv("2021_abstracts_with_keywords_categories.csv")
supra_category_list = np.unique(np.concatenate((df['Primary Category Parent Category'],df['Secondary Category Parent Category'])))
category_list = np.unique(np.concatenate((df['Primary Category'],df['Secondary Category'])))

categories = {}
categories_no_resp = set()
check_these = []
for category in category_list:
    categories[category.strip()] = {'category': category.strip(),
                            'Poster No': set()}  # use set to prevent duplication

for i, row in df.iterrows():
    for category in [row['Primary Category'], row['Secondary Category']]:
        if type(row['Poster No']) is list:
            for s in row['Poster No']:
                categories[category.strip()]['Poster No'].add(int(s))
        else:
            categories[category.strip()]['Poster No'].add(int(row['Poster No']))

cat_sets = {}
for s_cat in supra_category_list:
    cat_sets[s_cat] = {'categories': set()}

for i, row in df.iterrows():
    cat_sets[row['Primary Category Parent Category']]['categories'].add(row['Primary Category'].strip())
    cat_sets[row['Secondary Category Parent Category']]['categories'].add(row['Secondary Category'].strip())

with open("2021_categories_index_test.csv","w", encoding='utf8') as f:
    for supra_category in sorted(cat_sets):
        f.write("%s€\n" % (supra_category.strip()))
        for subcat in sorted(cat_sets[supra_category]['categories']):
            # also  REORDER SO THAT 'OTHER' APPEARS AT END OF SECTION
            f.write("%s€%s\n" % (subcat.strip(),",".join( str(e) for e in sorted(categories[subcat.strip()]['Poster No']))))            

# CURRENT MANUAL FIXES:
# fix neuro disorders chars
# REORDER SO THAT 'OTHER' APPEARS AT END OF SECTION

In [ ]:
# tex file needs to be run twice to incorporate cross-referenced links
!lualatex -interaction=nonstopmode ohbm2021_abstract_book.tex
!lualatex -interaction=nonstopmode ohbm2021_abstract_book.tex